# Read Dataset

In [2]:
import geopandas as gpd
import numpy as np
import polars as pl

# 读取 CSV 文件，过滤掉 MISSING_DATA 为 True 的行，按 TIMESTAMP 排序，选取 POLYLINE 列，取前 1000 个值
original_trajs = (
    pl.read_csv("./resource/dataset/Porto/porto_sample.csv")
    .filter(pl.col("MISSING_DATA") == False)
    .sort("TIMESTAMP")["POLYLINE"]
    .limit(1000)
    .map_elements(lambda x: np.array(eval(x)), return_dtype=pl.List(pl.Array(pl.Float64, 2)))
)
original_trajs.head(1)

POLYLINE
"list[array[f64, 2]]"
"[[-8.610291, 41.140746], [-8.6103, 41.140755], … [-8.60589, 41.145345]]"


In [3]:
import polars as pl
import numpy as np
from fedtraj.utils.trajclus import traclus_partition
from tqdm import tqdm


def cut_trajectorys_into_segments(original_trajs):
    new_data = []
    # 遍历 POLYLINE 列中的每个轨迹
    for traj in tqdm(original_trajs):
        # 调用 traj_clus 函数得到切分点布尔数组
        _, split_points = traclus_partition(traj)

        # 找到所有切分点的索引
        split_indices = np.where(split_points)[0]

        # 处理没有切分点的情况
        if len(split_indices) == 0:
            new_trajlen = len(traj)
            new_polyline = traj
            if new_trajlen > 0:  # 仅添加长度不为 0 的轨迹
                new_data.append([new_trajlen, new_polyline])
        else:
            # 切分轨迹
            for i in range(len(split_indices) - 1):
                start = split_indices[i]
                end = split_indices[i + 1]
                new_trajlen = end - start + 1
                new_polyline = traj[start: end + 1]
                if new_trajlen > 0:  # 仅添加长度不为 0 的轨迹
                    new_data.append([new_trajlen, new_polyline])

    # 转换为适合创建 Polars DataFrame 的格式
    trajlen_list = [item[0] for item in new_data]
    polyline_list = [item[1] for item in new_data]

    # 创建新的 Polars DataFrame
    new_df = pl.DataFrame({"POLYLINE": polyline_list})["POLYLINE"]
    return new_df


cut_trajs = cut_trajectorys_into_segments(original_trajs)

100%|██████████| 1000/1000 [00:02<00:00, 490.73it/s]


In [4]:
len(cut_trajs)

15686

In [5]:
from tqdm.contrib import tenumerate
from trajdl.datasets import Trajectory

all_trajs = [
    Trajectory(traj_pl.to_numpy(), entity_id=str(idx))
    for idx, traj_pl in tenumerate(cut_trajs, desc="transform trajectorys")
]
print(all_trajs[0], all_trajs[0].seq[:10])

transform trajectorys:   0%|          | 0/15686 [00:00<?, ?it/s]

Trajectory(entity_id=0, length=4) [[-8.610291 41.140746]
 [-8.6103   41.140755]
 [-8.610309 41.14089 ]
 [-8.613657 41.141358]]


In [6]:
len(all_trajs)

15686

In [7]:
import numpy as np
from tqdm import tqdm

def split_trajectories(all_trajs, min_length_train_val=2, max_length_train_val=64, min_length_test=2, max_length_test=64):
    """
    此函数用于将轨迹数据划分为训练集、验证集和测试集。

    参数:
    all_trajs (list): 包含所有轨迹数据的列表。
    min_length_train_val (int): 训练集和验证集轨迹的最小长度。
    max_length_train_val (int): 训练集和验证集轨迹的最大长度。
    min_length_test (int): 测试集轨迹的最小长度。
    max_length_test (int): 测试集轨迹的最大长度。

    返回:
    tuple: 包含训练集、验证集和测试集的元组。
    """
    # 获取所有轨迹的长度
    length = len(all_trajs)
    # 计算训练集、验证集和测试集的大小
    num_train = int(0.8 * length)
    num_val = int(0.1 * length)
    num_test = length - num_train - num_val

    # 初始化训练集、验证集和测试集列表
    train_traj, val_traj, test_traj = [], [], []

    # 划分训练集
    for traj in tqdm(all_trajs[:num_train], desc="Constructing training set"):
        # 检查轨迹长度是否在训练集要求的范围内
        if min_length_train_val <= len(traj) <= max_length_train_val:
            train_traj.append(traj)

    # 划分验证集
    for traj in tqdm(all_trajs[num_train:num_train + num_val], desc="Constructing validation set"):
        # 检查轨迹长度是否在验证集要求的范围内
        if min_length_train_val <= len(traj) <= max_length_train_val:
            val_traj.append(traj)

    # 划分测试集
    for traj in tqdm(all_trajs[num_train + num_val:], desc="Constructing test set"):
        # 检查轨迹长度是否在测试集要求的范围内
        if min_length_test <= len(traj) <= max_length_test:
            test_traj.append(traj)

    return train_traj, val_traj, test_traj

# 假设 all_trajs 是包含所有轨迹数据的列表
# all_trajs = [...]

# 调用函数进行数据划分
train_traj, val_traj, test_traj = split_trajectories(all_trajs)

# 打印各数据集的长度
print(f"Training set length: {len(train_traj)}")
print(f"Validation set length: {len(val_traj)}")
print(f"Test set length: {len(test_traj)}")

Constructing test set: 100%|██████████| 1570/1570 [00:00<00:00, 1225582.97it/s]

Training set length: 12548
Validation set length: 1568
Test set length: 1570


# Build Model

In [8]:
from trajdl import trajdl_cpp

# 基于经纬度系统的区域边界
boundary_original = trajdl_cpp.RectangleBoundary(
    min_x=-8.690261,
    min_y=41.140092,
    max_x=-8.549155,
    max_y=41.185969,
)
# 转换为基于平面坐标系的区域边界
boundary = boundary_original.to_web_mercator()
print(f"boundary_original: {boundary_original}")
print(f"boundary: {boundary}")

boundary_original: RectangleBoundary(min_x=-8.690261, min_y=41.140092, max_x=-8.549155, max_y=41.185969)
boundary: RectangleBoundary(min_x=-967395.429381, min_y=5033027.213480, max_x=-951687.581313, max_y=5039810.867670)


In [9]:
print(trajdl_cpp.convert_gps_to_webmercator(-8.690261, 41.140092))
print(trajdl_cpp.convert_gps_to_webmercator(0, 0))

WebMercatorCoord(x=-967395.4293806, y=5033027.2134798)
WebMercatorCoord(x=0.0000000, y=-0.0000000)


In [10]:
from trajdl.grid import SimpleGridSystem

# 网格的划分距离为100m
grid_width, grid_height = 100, 100

# 创建网格系统
grid = SimpleGridSystem(
    # 使用波尔多市的左下角点和右上角点来构建和切分网格系统
    boundary,
    step_x=grid_width,
    step_y=grid_height,
)
print(len(grid), grid.num_x_grids, grid.num_y_grids)

/home/niuyiming/.conda/envs/trajlib/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


10744 158 68


In [11]:
# 转墨卡托坐标系统
web_mercator_location = trajdl_cpp.convert_gps_to_webmercator(-8.610291, 41.140746)
# 转网格id
x, y = web_mercator_location.x, web_mercator_location.y
grid_id = grid.locate_unsafe(x, y)

print(web_mercator_location, grid_id)

WebMercatorCoord(x=-958493.2097019, y=5033123.8845711) 89


In [12]:
import os

from trajdl.tokenizers.t2vec import T2VECTokenizer

output_folder = "./output/t2vec"
os.makedirs(output_folder, exist_ok=True)

tokenizer = T2VECTokenizer.build(
    grid=grid,
    boundary=boundary_original,
    trajectories=all_trajs,
    max_vocab_size=40000,  # 词表支持的词元上限，排序逻辑是词元在数据集中的频率
    min_freq=64,  # 被命中至少min_freq次的网格称之为`hot cell`，tokenizer中仅保留`hot cell`
    with_kd_tree=True,
)
tokenizer.save_pretrained(os.path.join(output_folder, "tokenizer.pkl"))
print("num vocab: ", len(tokenizer))

num vocab:  113


In [13]:
from trajdl.common.enum import TokenEnum

k = 10  # 10个最近的网格
SPECIAL_TOKENS = TokenEnum.values()
vocab_list = tokenizer.vocab.keys()  # 获取全部字典的Token
loc_list, idx_list = zip(
    *((loc, tokenizer.loc2idx(loc)) for loc in vocab_list if loc not in SPECIAL_TOKENS)
)  # 剔除special tokens字

In [14]:
import numpy as np

dists, locations = tokenizer.k_nearest_hot_loc(
    loc_list, k=k
)  # 获取k个最近的网格以及对应的距离

# (num_locations, k)，索引矩阵
V = np.zeros(shape=(len(vocab_list), k), dtype=np.int64)

# (num_locations, k)，距离矩阵
D = np.zeros_like(V, dtype=np.float32)
D[idx_list, :] = dists

# 对于SPECIAL TOKENS，最近的token设定为自己
for token in SPECIAL_TOKENS:
    idx = tokenizer.loc2idx(token)
    V[idx] = idx

for line_idx, loc_list in zip(idx_list, locations):
    V[line_idx] = [tokenizer.loc2idx(loc) for loc in loc_list]

np.save(os.path.join(output_folder, "knn_indices.npy"), V)  # 保存k近邻网格的索引
np.save(os.path.join(output_folder, "knn_distances.npy"), D)  # 保留k近邻网格的距离

In [15]:
from trajdl.datasets import TrajectoryDataset
from trajdl.datasets.modules.t2vec import T2VECDataModuleV2

# 将Trajectory转换为TrajectoryDataset
train_dataset = TrajectoryDataset.init_from_trajectories(train_traj)
val_dataset = TrajectoryDataset.init_from_trajectories(val_traj)
test_dataset = TrajectoryDataset.init_from_trajectories(test_traj)

data_module = T2VECDataModuleV2(
    tokenizer=tokenizer,
    train_table=train_dataset,
    val_table=val_dataset,
    test_table=test_dataset,
    train_batch_size=4,
    val_batch_size=4,
    num_train_batches=10,
    num_val_batches=10,
    num_cpus=-1,
    k=2,
)

In [16]:
data_module.setup("fit")
train_dataloader = data_module.train_dataloader()
next(iter(train_dataloader))

T2VECSample(src=tensor([[110, 112, 112, 112],
        [110, 112, 112, 112],
        [ 44,  44,  44, 112],
        [ 44,  44,  44,  44]]), lengths=(1, 1, 3, 4), target=tensor([[108, 110, 109, 112, 112],
        [108, 110, 109, 112, 112],
        [108,  44,  44,  44, 109],
        [108,  44,  44,  44, 109]]))

In [17]:
from trajdl.algorithms.t2vec import T2VEC

# 构建模型，我们使用默认参数，用户也可以根据文档修改模型的类型，比如使用GRU、LSTM等编码器
model = T2VEC(
    embedding_dim=256,
    hidden_size=256,
    tokenizer=tokenizer,
    knn_indices_path=os.path.join(output_folder, "knn_indices.npy"),
    knn_distances_path=os.path.join(output_folder, "knn_distances.npy"),
)
model

T2VEC(
  (embedding): SimpleEmbedding(
    (embedding): Embedding(113, 256, padding_idx=112)
  )
  (encoder): T2VECEncoder(
    (emb): SimpleEmbedding(
      (embedding): Embedding(113, 256, padding_idx=112)
    )
    (encoder): GRU(256, 256, batch_first=True)
  )
  (decoder): DecoderWithAttention(
    (embedding_layer): SimpleEmbedding(
      (embedding): Embedding(113, 256, padding_idx=112)
    )
    (rnn): StackingGRU(
      (grus): ModuleList(
        (0): GRUCell(256, 256)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (attention): GlobalAttention(
      (L1): Linear(in_features=256, out_features=256, bias=False)
      (L2): Linear(in_features=512, out_features=256, bias=False)
      (softmax): Softmax(dim=1)
      (tanh): Tanh()
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (projector): Sequential(
    (0): Linear(in_features=256, out_features=113, bias=True)
    (1): LogSoftmax(dim=1)
  )
  (loss_fn): KLDivLoss()
)

In [18]:
import lightning as L

trainer = L.Trainer(max_epochs=8, logger=False, enable_checkpointing=False)
trainer.fit(model, data_module)

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type                 | Params | Mode 
-----------------------------------------------------------
0 | embedding | SimpleEmbedding      | 28.9 K | train
1 | encoder   | T2VECEnco

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


In [19]:
import torch

# 先获取test_dataloader
data_module.setup("test")
test_dataloader = data_module.test_dataloader()
test_sample_1 = next(iter(test_dataloader))
test_sample_2 = next(iter(test_dataloader))

model.eval()

with torch.inference_mode():
    vec_1 = model(test_sample_1)
    vec_2 = model(test_sample_2)

batch_vec_1, batch_vec_2 = vec_1.detach().cpu().numpy(), vec_2.detach().cpu().numpy()
print(batch_vec_1.shape, batch_vec_2.shape)

(4, 256) (4, 256)


In [20]:
from sklearn.metrics.pairwise import euclidean_distances

print(euclidean_distances(batch_vec_1, batch_vec_2))

[[6.070461  2.5555851 7.8009853 6.837785 ]
 [6.4445977 2.4364927 7.777173  6.776062 ]
 [6.070461  2.5555851 7.8009853 6.837785 ]
 [1.3182648 6.4440093 8.017162  7.0369325]]


In [21]:
from trajdl.datasets.modules.t2vec import generate_samples
from trajdl.common.samples import T2VECSample
from torch.nn.utils.rnn import pad_sequence
import torch
from tqdm import tqdm

# 假设 batch_size 是我们指定的批量大小
batch_size = 32

samples = []

for traj in all_trajs:
    sample = generate_samples(
        tokenizer, traj
    )
    samples.append(sample)

all_outputs = []

# 按 batch_size 对 samples 进行分批处理
for start_idx in tqdm(range(0, len(samples), batch_size), desc="calculating embedding"):
    end_idx = min(start_idx + batch_size, len(samples))
    batch_samples = samples[start_idx:end_idx]
    src_samples, src_lens, trg_samples = zip(*batch_samples)

    # 创建当前批次的 T2VECSample 对象
    batch = T2VECSample(
        src=pad_sequence(src_samples, batch_first=True, padding_value=tokenizer.pad),
        lengths=src_lens,
        target=pad_sequence(trg_samples, batch_first=True, padding_value=tokenizer.pad),
    )

    # 进行推理
    batch_output = model(batch)

    # 将当前批次的输出添加到 all_outputs 列表中
    all_outputs.append(batch_output)

# 将所有批次的输出拼接起来得到完整的结果
all_embeddings = torch.cat(all_outputs, dim=0)

print(all_embeddings.shape, len(all_trajs), len(samples))

calculating embedding: 100%|██████████| 491/491 [00:01<00:00, 321.45it/s]


torch.Size([15686, 256]) 15686 15686


In [22]:
import pickle


with open("./resource/results/t2vec/trajs.pkl", 'wb') as f:
    pickle.dump(all_trajs, f)

# 使用 torch 保存嵌入
torch.save(all_embeddings, "./resource/results/t2vec/embs.pkl")

# Embedding Clustering

In [24]:
import os
import torch
import pandas as pd
import pickle
from easydict import EasyDict
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np
from math import sqrt, atan2, sin


class Segment(EasyDict):
    def __init__(self, segment_id, points, emb):
        super().__init__()
        self.id = segment_id
        self.points = points
        self.emb = emb


class Cluster:
    def __init__(self, segments):
        self.items = segments
        self.size = len(segments)
        self.centroid = self._calculate_centroid()
        self.radius = self._calculate_radius()
        self.merged = False

    def _calculate_centroid(self):
        total_x = 0
        total_y = 0
        for seg in self.items:
            start, end = seg.points[0], seg.points[-1]
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2
            total_x += mid_x
            total_y += mid_y
        centroid_x = total_x / self.size
        centroid_y = total_y / self.size
        return (centroid_x, centroid_y)

    def _calculate_radius(self):
        max_distance = 0
        for seg in self.items:
            start, end = seg.points[0], seg.points[-1]
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2
            distance = self.compute_point_distance((mid_x, mid_y), self.centroid)
            if distance > max_distance:
                max_distance = distance
        return max_distance

    @staticmethod
    def compute_point_distance(p1, p2):
        return sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)


def compute_angular_distance(seg1, seg2):
    start1, end1 = seg1.points[0], seg1.points[-1]
    start2, end2 = seg2.points[0], seg2.points[-1]
    vector1 = (end1[0] - start1[0], end1[1] - start1[1])
    vector2 = (end2[0] - start2[0], end2[1] - start2[1])
    angle1 = atan2(vector1[1], vector1[0])
    angle2 = atan2(vector2[1], vector2[0])
    angle_diff = abs(angle1 - angle2)
    if angle_diff > np.pi:
        angle_diff = 2 * np.pi - angle_diff
    len1 = Cluster.compute_point_distance(start1, end1)
    len2 = Cluster.compute_point_distance(start2, end2)
    return abs(sin(angle_diff)) * max(len1, len2)


def compute_vector_distance(r1, r2):
    sum_square = torch.sum((r1 - r2) ** 2)
    return torch.sqrt(sum_square).item()


def calculate_distance(seg1, seg2, alpha, beta, gamma):
    d1 = Cluster.compute_point_distance(
        seg1.points[0], seg2.points[0]
    ) + Cluster.compute_point_distance(seg1.points[-1], seg2.points[-1])
    d2 = compute_angular_distance(seg1, seg2)
    d3 = compute_vector_distance(seg1.emb, seg2.emb)
    return alpha * d1 + beta * d2 + gamma * d3


def local_clustering(trajectory_segments, eps, min_samples, alpha, beta, gamma):
    num_segments = len(trajectory_segments)
    distance_matrix = np.zeros((num_segments, num_segments))
    # 创建 tqdm 进度条
    total_iterations = num_segments * (num_segments - 1) // 2
    progress_bar = tqdm(total=total_iterations, desc="计算距离矩阵")
    count = 0
    for i in range(num_segments):
        for j in range(i + 1, num_segments):
            dist = calculate_distance(
                trajectory_segments[i], trajectory_segments[j], alpha, beta, gamma
            )
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist
            # 更新进度条
            count += 1
            progress_bar.update(1)

    db = DBSCAN(eps=eps, min_samples=min_samples, metric="precomputed")
    labels = db.fit_predict(distance_matrix)

    local_clusters = []
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            continue
        cluster_indices = np.where(labels == label)[0]
        cluster_segments = [trajectory_segments[i] for i in cluster_indices]
        cluster = Cluster(cluster_segments)
        local_clusters.append(cluster)

    return local_clusters


def run_clustering():
    trajs_filepath = r"./resource/results/t2vec/trajs.pkl"
    embs_filepath = r"./resource/results/t2vec/embs.pkl"

    with open(trajs_filepath, "rb") as f:
        all_trajs = pickle.load(f)

    # 使用 torch 加载嵌入
    all_embeddings = torch.load(embs_filepath)

    all_segments = [
        Segment(i, traj, emb)
        for i, (traj, emb) in enumerate(zip(all_trajs, all_embeddings))
    ]

    # 本地聚类参数
    local_eps = 1000.0
    local_min_samples = 2
    alpha = 1
    beta = 1
    gamma = 1

    # 进行集中式聚类
    local_clusters = local_clustering(
        all_segments, local_eps, local_min_samples, alpha, beta, gamma
    )

    # 计算轮廓系数
    all_seg = []
    labels = []
    for i, cluster in enumerate(local_clusters):
        for segment in cluster.items:
            all_seg.append(segment)
            labels.append(i)

    num_segments = len(all_seg)
    distance_matrix = np.zeros((num_segments, num_segments))
    for i in range(num_segments):
        for j in range(i + 1, num_segments):
            dist = calculate_distance(all_seg[i], all_seg[j], alpha, beta, gamma)
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist

    silhouette_avg = silhouette_score(distance_matrix, labels, metric="precomputed")
    print(f"Silhouette Coefficient: {silhouette_avg}")

    # 输出结果
    for i, cluster in enumerate(local_clusters):
        print(f"Cluster {i}:")
        print(f"  Centroid: {cluster.centroid}")
        print(f"  Radius: {cluster.radius}")
        print(f"  Size: {cluster.size}")


if __name__ == "__main__":
    run_clustering()

计算距离矩阵: 100%|██████████| 123017455/123017455 [1:34:56<00:00, 21594.81it/s]


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)